In [4]:
from pyspark.sql import SparkSession

In [3]:
spark.stop()
spark = (SparkSession
         .builder
         .appName("Python Spark SQL Hive")
         .config("spark.sql.warehouse.dir", "user/hive/warehouse")
         .config("hive.metastore.uris", "thrift://ip-10-0-4-11.eu-west-1.compute.internal:9083")
         .enableHiveSupport()
         .getOrCreate()
         )

In [3]:
spark.sql("describe questions").show(truncate=False)

AnalysisException: "Table or view 'questions' not found in database 'default';"

In [6]:
spark.sql("describe project_question_tags").show(truncate=False)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|id      |smallint |null   |
|tag     |string   |null   |
+--------+---------+-------+



In [7]:
df_q = spark.sql(""" SELECT * FROM project_questions """)
df_q.printSchema()

root
 |-- usuario: short (nullable = true)
 |-- creationdate: string (nullable = true)
 |-- closeddate: string (nullable = true)
 |-- deletiondate: string (nullable = true)
 |-- score: short (nullable = true)
 |-- owneruserid: short (nullable = true)
 |-- answercount: short (nullable = true)



In [8]:
df_qt = spark.sql(""" SELECT * FROM project_question_tags """)
df_qt.printSchema()

root
 |-- id: short (nullable = true)
 |-- tag: string (nullable = true)



In [9]:
df_qt.registerTempTable('usuario')

In [10]:
df_q.registerTempTable('fecha')

In [11]:
df_qt_tmp=spark.sql('select usuario.tag, count(*) as contador from usuario group by usuario.tag order by contador desc limit 20')
df_qt_tmp.show()

+-------------+--------+
|          tag|contador|
+-------------+--------+
|   javascript| 1649631|
|         java| 1563820|
|          php| 1365600|
|           c#| 1321027|
|      android| 1288338|
|       jquery| 1011324|
|       python|  854172|
|         html|  776512|
|          ios|  635227|
|          c++|  627341|
|        mysql|  561375|
|          css|  545587|
|          sql|  436546|
|      asp.net|  400619|
|  objective-c|  338539|
|ruby-on-rails|  325003|
|            c|  306418|
|         .net|  301454|
|       iphone|  274464|
|    angularjs|  267560|
+-------------+--------+



In [12]:
df_qt_tmp.registerTempTable('conteo')

In [13]:
from pyspark.sql.functions import col, unix_timestamp, to_date

In [29]:
df_c1=spark.sql("select t.tag, year(q.creation_date) as fecha, count(t.tag) as tecno FROM questions q, question_tags t where q.id = t.id and group by t.tag, year(q.creation_date) order by count(t.tag) desc, year(q.creation_date) desc")
df_c1.show(50)

+----------+-----+-------+
|       tag|fecha|  tecno|
+----------+-----+-------+
|javascript| 2016|2921008|
|javascript| 2015|2714680|
|javascript| 2014|2479792|
|      java| 2014|2411560|
|      java| 2015|2397848|
|      java| 2016|2346544|
|      java| 2013|2003048|
|javascript| 2013|1852464|
|       php| 2014|1850000|
|        c#| 2014|1781488|
|       php| 2013|1775080|
|       php| 2015|1760640|
|        c#| 2013|1700944|
|       php| 2016|1697112|
|    python| 2016|1677704|
|        c#| 2015|1658472|
|        c#| 2016|1654752|
|   android| 2014|1543544|
|   android| 2015|1508960|
|   android| 2016|1473928|
|   android| 2013|1408624|
|    python| 2015|1392408|
|      java| 2012|1387560|
|        c#| 2012|1329536|
|       php| 2012|1308080|
|javascript| 2012|1167392|
|    python| 2014|1165648|
|   android| 2012|1135432|
|        c#| 2011|1023040|
|       ios| 2015|1014352|
|       ios| 2014| 942456|
|    python| 2013| 900848|
|      java| 2011| 900544|
|       ios| 2016| 893984|
|

In [31]:
df_c1.write.format("csv").save('hdfs:///user/hive/warehouse/tecnofechajl.csv')
